In [1]:
# Imports
from IPython.display import HTML          
import pandas as pd
import torch
#from torch.utils.data import TensorDataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_metric, Dataset
from sklearn.model_selection import train_test_split


In [2]:
# Left align data in pd tables
HTML("""
<style>
    th, td {
        text-align: left !important;
    }
</style>
""")

In [3]:
# Import data and clean 
# Data is from https://www.kaggle.com/datasets/arshkandroo/behavioural-tweets?select=Lonely_Tweets.csv.
# Tweepy API was used to scrape twitter for lonely and normal tweets. The quality is okay but 
# some misclassification. The tweets are already cleaned using the NLTK library according to the
# data collectors.

df_normal = pd.read_csv('Data/Normal_Tweets.csv.xls')
del df_normal['Unnamed: 0']
df_normal.columns.values[0] = 'tweet'
df_normal['label'] = 0                                  # 0 for normal tweets 
df_normal['id'] = range(1, len(df_normal) + 1)

df_lonely = pd.read_csv('Data/Lonely_Tweets.csv.xls')
del df_lonely['Unnamed: 0']
df_lonely.columns.values[0] = 'tweet'
df_lonely['label'] = 1                                  # 1 for lonely tweets
df_lonely['id'] = range(len(df_normal) + 1, len(df_normal) + len(df_lonely) + 1)

df = pd.concat([df_normal, df_lonely])

In [4]:
# Display summary data
print(f"The data set has {len(df)} tweets in it.")
print()

print(f"There are {df.shape[1]} variables and they are as follows: ")
print(df.dtypes)
print()

print("The first and last five rows of the table are:")
pd.concat([df.head(), df.tail()])


The data set has 18447 tweets in it.

There are 3 variables and they are as follows: 
tweet    object
label     int64
id        int64
dtype: object

The first and last five rows of the table are:


,tweet,label,id
0,remember hillary email non secure server,0,1
1,cant avoid demon,0,2
2,por fin la pusieron en spotify losing way de f...,0,3
3,kills,0,4
4,thank introduce important gunsense law make co...,0,5
8517,love son get paternity test person sex doesnt ...,1,18443
8518,tell father dont want bother need talk god cau...,1,18444
8519,lrt leave alone finally found need cry love ha...,1,18445
8520,know something dont need one love want want wa...,1,18446
8521,dont need want meet want several moment side,1,18447


In [5]:
# Split into training and validation set
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [6]:
# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
train_texts = train_df.tweet.tolist()
train_labels = train_df.label.tolist()

labels = torch.tensor(train_labels)

encoded_inputs = tokenizer(
    train_texts,
    add_special_tokens=True,
    truncation=True,
    max_length=20,
    padding='max_length',
    return_attention_mask=True,
    return_tensors="pt",
)

input_ids = encoded_inputs["input_ids"]
attention_masks = encoded_inputs["attention_mask"]


In [8]:
train_data_dict = {
    'input_ids': input_ids,
    'attention_mask': attention_masks,
    'labels': labels
}

train_data_dict = {key: tensor.to("mps") for key, tensor in train_data_dict.items()}
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)

# Create a Dataset object
train_dataset = Dataset.from_dict(train_data_dict)

In [9]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    logging_dir='./logs',
    logging_steps=10,
    save_strategy="epoch",
    disable_tqdm=True,  
    log_level="error" 
)

trainer = Trainer(
    model=model,  
    args=training_args,
    train_dataset=train_dataset,  
    # Optionally add a validation dataset
    # eval_dataset=validation_dataset,
)

In [10]:
trainer.train()

{'loss': 0.6791, 'grad_norm': 3.446047782897949, 'learning_rate': 4.945828819068256e-05, 'epoch': 0.010834236186348862}
{'loss': 0.5427, 'grad_norm': 6.040877819061279, 'learning_rate': 4.891657638136511e-05, 'epoch': 0.021668472372697724}
{'loss': 0.4229, 'grad_norm': 3.940645456314087, 'learning_rate': 4.837486457204768e-05, 'epoch': 0.032502708559046585}
{'loss': 0.3626, 'grad_norm': 8.920709609985352, 'learning_rate': 4.7833152762730235e-05, 'epoch': 0.04333694474539545}
{'loss': 0.3025, 'grad_norm': 6.809895992279053, 'learning_rate': 4.7291440953412786e-05, 'epoch': 0.05417118093174431}
{'loss': 0.3584, 'grad_norm': 4.547197341918945, 'learning_rate': 4.6749729144095344e-05, 'epoch': 0.06500541711809317}
{'loss': 0.3535, 'grad_norm': 5.68408727645874, 'learning_rate': 4.6208017334777896e-05, 'epoch': 0.07583965330444203}
{'loss': 0.2832, 'grad_norm': 1.9386407136917114, 'learning_rate': 4.5666305525460454e-05, 'epoch': 0.0866738894907909}
{'loss': 0.2697, 'grad_norm': 9.288897514

TrainOutput(global_step=923, training_loss=0.25718017708699853, metrics={'train_runtime': 125.2839, 'train_samples_per_second': 117.789, 'train_steps_per_second': 7.367, 'train_loss': 0.25718017708699853, 'epoch': 1.0})

In [11]:
# model.save_pretrained("bert_model")
# tokenizer.save_pretrained("bert_model")